In [59]:
import numpy as np
from scipy.spatial import Delaunay as Del
import matplotlib.pyplot as plt
import pandas as pd
import sys
from sklearn.decomposition import PCA
from scipy.spatial import ConvexHull
from sklearn.manifold import TSNE
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt

In [60]:
# grid = np.random.randn(200, 2)* 20
grid = []
for i in range(50):
    for j in range(20):
        grid.append([i, j])
grid = np.array(grid)/5.0

prev = np.zeros((grid.shape[0], grid.shape[0]))

In [61]:
def  triangulate(grid):
    adj = np.zeros((grid.shape[0], grid.shape[0]))
    tri = Del(grid)
    chull = ConvexHull(grid).simplices
#     print chull
    for simplegrid in tri.simplices:
        for vert in simplegrid:
            for vert2 in simplegrid:
                adj[vert][vert2] = 1
#     for line in chull:
#         adj[line[0]][line[1]]=0
#         adj[line[1]][line[0]]=0
    
    return adj

In [62]:
# plt.triplot(grid.T[0], grid.T[1], tri.simplices.copy())
# plt.plot(grid.T[0], grid.T[1], 'o')
# for i in range(grid.shape[0]):
#     plt.text(grid.T[0][i], grid.T[1][i], i)
# plt.show()

In [63]:
def get_neighbors(point, grid, r):
    
    if grid.any():
        G = triangulate(grid)  
    else:
        G = r
    return np.where(G[point])[0], G
    print np.where(np.linalg.norm(grid - grid[point], axis =1) < r) [0]
#     return np.where(np.linalg.norm(grid - grid[point], axis = 1) < r) [0]

In [64]:
X = np.array(pd.read_csv('/home/senanayaked/data/4C_2M.csv', header=None))
X = np.append(X, np.random.randn(X.shape[0], 1), axis =1)#[:, :2]#np.random.random((100, 3))
X += abs(X.min())
X /= X.max()
# X = np.random.random((200, 3))
C = np.ones((X.shape[0], X.shape[1]))*X.mean()#np.random.random((X.shape[0], X.shape[1]))
# grid = np.random.random((X.shape[0], 2))*20
# grid = X[:, :2]
# plt.scatter(grid.T[0], grid.T[1])
# plt.show()
print X.shape

(1000, 3)


In [65]:
# try:

ts = 0.0001
fs = 0.0001


for it in range(250):
    sys.stdout.write('\r iteration %s'% str(it+1))
    for x in X[np.random.permutation(X.shape[0])]:
        # find BMU    
        bmu = np.argmin(np.linalg.norm(C -x, axis = 1))
        p = grid
        if ts*1.0 / (ts+fs) > 0.75:
            p = np.zeros(grid.shape)
        neighbors, G = get_neighbors(bmu, p, prev)
        if np.all(G==prev):
            ts += 1
        else:
            fs += 1

        prev=G
        
        
        
#             print neighbors
        dists = np.linalg.norm(grid[neighbors] - grid[bmu], axis = 1)
        moving_amounts = 0.75*np.array([np.exp(-dists)]).T/dists.shape[0]
#         print moving_amounts

        C[neighbors] += 1*(x- C[neighbors]) * np.exp(-it/500) #* moving_amounts

        # move neighbors closer


        moving_directions = grid[bmu] - grid[neighbors]

        grid[neighbors] += np.exp(-it/100)*moving_amounts * moving_directions


        not_neighbors = np.setdiff1d(np.array(range(X.shape[0])), neighbors)

        dists = np.linalg.norm(grid[not_neighbors] - grid[bmu], axis = 1)
        moving_amounts = 0.75*np.array([np.exp(-dists)]).T//dists.shape[0]
        moving_directions = grid[bmu] - grid[not_neighbors]
        grid[not_neighbors] -= np.exp(-it/100)* moving_amounts * moving_directions
# except Exception:
#     print Exception.message
#     pass
#predict 
out = []
for x in X:
    bmu = np.argmin(np.linalg.norm(C -x, axis = 1))
    out.append(grid[bmu])
out = np.array(out)

 iteration 250

In [66]:
plt.scatter(out.T[0], out.T[1], c=np.array(range(X.shape[0]))/250, edgecolors='none', s= 50, alpha=0.5)
# plt.plot(out.T[0], out.T[1], c='grey', alpha=0.5)
plt.show()

In [67]:
t = PCA(2).fit_transform(X)
plt.scatter(out.T[0], out.T[1], plt.cm.Set1(np.array(range(X.shape[0])).astype(float)/X.shape[0]), edgecolors='none', alpha=0.5,s=50)
#plt.cm.Set1(np.array(range(X.shape[0])).astype(float)/X.shape[0]), edgecolors='none', alpha=0.5)
# plt.plot(t.T[0], t.T[1], c='grey', alpha=0.5)
plt.show()

TypeError: scatter() got multiple values for keyword argument 's'

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
ax.scatter3D(X.T[0], X.T[1], X.T[2], c=plt.cm.Set1(np.array(range(X.shape[0])).astype(float)/X.shape[0]),s = 50, edgecolors='none', alpha=0.5)
# plt.scatter(X, Y, Z, c = plt.cm.Set1(theta/200), edgecolors='none', alpha = 0.7, s = 50)
plt.show()

In [ ]:
X.shape